In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest,SelectPercentile, SelectFpr,SelectFdr,GenericUnivariateSelect,chi2,f_regression

In [ ]:
%matplotlib inline

In [ ]:
p_num = 1
original_df = pd.read_csv(r'C:\Users\james\Documents\Hishtalmut_Kamdan\Targil 1 - Data Visualiztion\My - Home Credit Default Risk.csv')
original_df.drop(columns=['Unnamed: 0'], inplace=True)

## Remove NAN

In [ ]:
df_preprocessed = original_df.fillna(method='ffill')

In [ ]:
df_preprocessed[df_preprocessed['CNT_CHILDREN'].isnull()] = df_preprocessed[df_preprocessed['CNT_CHILDREN'].isnull()].fillna(0)

## Remove lines with target = 2

In [ ]:
target_2_index_list = list(df_preprocessed[df_preprocessed['TARGET']==2].index)

In [ ]:
df_clean_target = df_preprocessed.drop(df_preprocessed.index[target_2_index_list])

In [ ]:
df_clean_target['TARGET'].value_counts()

## Transform objects to int

In [ ]:
def get_object_column_names(df):
    types_df = df.dtypes.reset_index()
    cols_list = list(types_df['index'][types_df[0]=='object'])
    return cols_list

In [ ]:
df_dummies = df_clean_target.copy()

In [ ]:
for col in get_object_column_names(df_dummies):
    df_dummies = pd.concat([df_dummies.drop(col,axis=1), pd.get_dummies(df_dummies[col], prefix=col)], axis=1)

## Remove all columns with negative values

In [ ]:
cols_to_remove = []
for col in list(df_dummies.columns):
    if len(df_dummies[col][df_dummies[col] < 0]) > 0:
        cols_to_remove.append(col)

In [ ]:
cols_to_remove

In [ ]:
df_for_XY = df_dummies.drop(cols_to_remove, axis=1)

## set X and y variables

In [ ]:
X = df_for_XY.drop('TARGET', axis=1)

In [ ]:
y = df_for_XY['TARGET']

## Feature Selection - Filter Methods - Univariate feature selection

### Chi squared test - SelectKBest

In [ ]:
SB = SelectKBest(chi2, k=2)
x_chi = SB.fit_transform(X, y)

In [ ]:
column_names = X.columns[SB.get_support()]

In [ ]:
column_names

### Chi squared test - SelectPercentile

In [ ]:
SP = SelectPercentile(chi2, percentile=2)
x_chi = SP.fit_transform(X, y)

In [ ]:
column_names = X.columns[SB.get_support()]

In [ ]:
column_names

### Chi squared test - GenericUnivariateSelect - mode : {‘percentile’, ‘k_best’, ‘fpr’, ‘fdr’, ‘fwe’}

In [ ]:
transformer = GenericUnivariateSelect(chi2, 'fwe', param=0.1)
x_chi = transformer.fit_transform(X, y)

In [ ]:
column_names = X.columns[transformer.get_support()]

In [ ]:
column_names

### Correlations

In [ ]:
p_num = 0

In [ ]:
# remove FLAG_MOBIL column
X = df_for_XY.drop(['TARGET','FLAG_MOBIL'], axis=1)

#### Correlation between features

In [ ]:
corr = X.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
selected_columns = []
for i in range(corr.shape[0]):
    for j in range(corr.shape[1]):
        if j<i and corr.iloc[j][i]>0.5:
            selected_columns.append((list(corr.columns)[i],corr.iloc[j].name))

In [ ]:
selected_columns

#### correlation between label and features

In [ ]:
# corr_label = df_for_XY.drop('FLAG_MOBIL',axis=1).corr()['TARGET'].to_frame()
# corr_label.sort_values('TARGET',inplace=True)
# plt.figure(p_num)
# sns.heatmap(corr_label, annot=True, annot_kws={"size": 10})
# plt.suptitle("correlation between label and features", fontsize=20)
# p_num+=1

In [ ]:
corr_label_df = corr_label.abs().reset_index()

In [ ]:
list_of_top_matches = list(corr_label_df['index'][corr_label_df['TARGET']>0.15])

In [ ]:
list_of_top_matches

In [ ]:
new_corr_label = df_for_XY[list_of_top_matches].corr()['TARGET'].to_frame().abs()
new_corr_label.sort_values('TARGET',inplace=True)
plt.figure(p_num)
sns.heatmap(new_corr_label, annot=True, annot_kws={"size": 10})
plt.suptitle("correlation between label and features", fontsize=20)
p_num+=1